<a href="https://colab.research.google.com/github/vperng/AAI520-NPL-Chatbot/blob/T5-Training/Chatbot_Project_Team6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Generative Chatbot Design

Rene Ortiz, Vivian Perng, Karthik Raghavan

## Project Overview

- Goal: Build a chatbot that can carry out multi-turn conversations, adapt to context, and handle a variety of topics.
- Output: A web or app interface where users can converse with the chatbot.


## Data Collection and Preprocessing

In [51]:
### COMMENT OUT IF NOT USING GOOGLE COLAB ####
import os
from google.colab import drive
drive.mount('/content/drive')

# Set the path to project folder
project_folder = '/content/drive/MyDrive/Colab Notebooks/Chatbot_Project'

# Change the working directory to project folder
os.chdir(project_folder)

# Check the current working directory
print(os.getcwd())

# Install packages that are not on Google Colab here
!pip install datasets
!pip install evaluate
!pip install rouge_score

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Chatbot_Project
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=359261657c77d8224b13f54b6af27d43d055c9124104a9484beb81a215cc00b7
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [49]:
# Import libraries
import json
import numpy as np
import pandas as pd
import torch
from transformers import T5TokenizerFast, T5ForConditionalGeneration, Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from evaluate import load

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


Using device: cuda


In [3]:
# Open files for reading
with open('/content/drive/MyDrive/Colab Notebooks/Chatbot_Project/train-v1.1.json', 'r') as file:
    train_data = json.load(file)
print(type(train_data))

<class 'dict'>


In [4]:
# Determine type for each key
for key in train_data.keys():
    print(f'Type of value for key "{key}": {type(train_data[key])}')

# Print information about each key
for key in train_data.keys():
    value = train_data[key]
    if isinstance(value, list):
        print(f'Length of list for key "{key}": {len(value)}')
    else:
        print(f'Value for key "{key}": {value}')

Type of value for key "data": <class 'list'>
Type of value for key "version": <class 'str'>
Length of list for key "data": 442
Value for key "version": 1.1


In [5]:
def explore_record_path(data, path=None):
    """
    Recursively explore the nested structure of a JSON-like object (dict or list) and print the paths
    and types at each level.

    Parameters:
    data (dict or list): The JSON data or a nested structure to explore.
    path (list): A list to track the current record path (used internally for recursion).

    Example usage:
    explore_record_path(train_data['data'])
    """

    if path is None:
        path = []

    # If the data is a dictionary, print the keys and recursively explore
    if isinstance(data, dict):
        for key in data:
            print(f"{' > '.join(path + [key])}: {type(data[key])}")
            explore_record_path(data[key], path + [key])

    # If the data is a list, print the index and recursively explore the first element
    elif isinstance(data, list) and len(data) > 0:
        print(f"{' > '.join(path)}[]: {type(data[0])}")
        explore_record_path(data[0], path)

# Explore the nested JSON structure to determine record path
explore_record_path(train_data['data'])

[]: <class 'dict'>
title: <class 'str'>
paragraphs: <class 'list'>
paragraphs[]: <class 'dict'>
paragraphs > context: <class 'str'>
paragraphs > qas: <class 'list'>
paragraphs > qas[]: <class 'dict'>
paragraphs > qas > answers: <class 'list'>
paragraphs > qas > answers[]: <class 'dict'>
paragraphs > qas > answers > answer_start: <class 'int'>
paragraphs > qas > answers > text: <class 'str'>
paragraphs > qas > question: <class 'str'>
paragraphs > qas > id: <class 'str'>


In [6]:
def squad1_json_to_dataframe(file_path, record_path):
    """
    Function to convert the dataset JSON file to a Pandas DataFrame.

    file_path (str): Path to the JSON file
    record_path (list): Path to the deepest level in the JSON structure (default is ['data', 'paragraphs', 'qas', 'answers']).

    Returns dataFrame containing the parsed data.
    """
    # Load JSON data
    with open(file_path, 'r') as f:
        file_data = json.load(f)

    # Extract and normalize the nested JSON structures
    answers_df = pd.json_normalize(file_data, record_path)
    questions_df = pd.json_normalize(file_data, record_path[:-1])
    paragraphs_df = pd.json_normalize(file_data, record_path[:-2])

    # Add 'answer start' and 'text' from answers_df to questions_df
    questions_df['answer_start'] = answers_df['answer_start']
    questions_df['answers'] = answers_df['text']

    # Create 'context' by repeating the corresponding paragraph for each question
    questions_df['context'] = np.repeat(paragraphs_df['context'].values, paragraphs_df.qas.str.len())

    # Create final DataFrame with necessary columns
    data = questions_df[['id', 'question', 'context', 'answer_start', 'answers']].copy()

    # Add 'c_id' to uniquely identify each context
    data['c_id'] = pd.factorize(data['context'])[0]

    return data.reset_index(drop=True)


In [7]:
# loading the data
file_path = '/content/drive/MyDrive/Colab Notebooks/Chatbot_Project/train-v1.1.json'
df = squad1_json_to_dataframe(file_path, record_path=['data', 'paragraphs', 'qas', 'answers'])
df

,id,question,context,answer_start,answers,c_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",279,the Main Building,0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",381,a Marian place of prayer and reflection,0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",92,a golden statue of the Virgin Mary,0
...,...,...,...,...,...,...
87594,5735d259012e2f140011a09d,In what US state did Kathmandu first establish...,"Kathmandu Metropolitan City (KMC), in order to...",229,Oregon,18890
87595,5735d259012e2f140011a09e,What was Yangon previously known as?,"Kathmandu Metropolitan City (KMC), in order to...",414,Rangoon,18890
87596,5735d259012e2f140011a09f,With what Belorussian city does Kathmandu have...,"Kathmandu Metropolitan City (KMC), in order to...",476,Minsk,18890
87597,5735d259012e2f140011a0a0,In what year did Kathmandu create its initial ...,"Kathmandu Metropolitan City (KMC), in order to...",199,1975,18890


In [8]:
def add_end_position(start_ix, answer):
  """
  This function determines the end position of each answer
  """
  start_position = start_ix
  end_position = start_position + len(answer)
  return end_position

In [9]:
# add end positions to the dataset
df['answer_end'] = df.apply(lambda x: add_end_position(x['answer_start'], x['answers']), axis=1)
df.head()

,id,question,context,answer_start,answers,c_id,answer_end
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0,541
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0,213
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",279,the Main Building,0,296
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",381,a Marian place of prayer and reflection,0,420
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",92,a golden statue of the Virgin Mary,0,126


## Data Exploration


In [10]:
# Initialize tokenizer for Flan-T5-Base
tokenizer = T5TokenizerFast.from_pretrained('google/flan-t5-base')

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
def count_tokens(text, tokenizer):
    df[f'tokens_{text}'] = df[text].apply(lambda x: len(tokenizer.tokenize(x)))

    # Find the maximum number of tokens in the context column
    max_tokens = df[f'tokens_{text}'].max()

    print(f"The maximum number of tokens in the {text} column is: {max_tokens}")

count_tokens('context', tokenizer)
count_tokens('question', tokenizer)
count_tokens('answers', tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (625 > 512). Running this sequence through the model will result in indexing errors


The maximum number of tokens in the context column is: 967
The maximum number of tokens in the question column is: 66
The maximum number of tokens in the answers column is: 75


## Text Preprocessing

In [12]:
def chunk_context(question_tokenized, context_tokenized, answer_start, answer_end, tokenizer, max_length=510):
    input_ids = context_tokenized['input_ids']
    offsets = context_tokenized['offset_mapping']

    # Find token-level start and end positions of the answer
    token_start_position = next(idx for idx, offset in enumerate(offsets) if offset[0] >= answer_start)
    token_end_position = next(idx for idx, offset in enumerate(offsets) if offset[1] >= answer_end)

    # Calculate available space for context (subtracting question length)
    available_context_length = max_length - len(question_tokenized['input_ids'])

    # Determine how much context to keep around the answer
    half_window_size = (available_context_length - (token_end_position - token_start_position)) // 2
    start_window = max(token_start_position - half_window_size, 0)
    end_window = min(token_end_position + half_window_size, len(input_ids))

    # Extract the relevant context chunk
    truncated_input_ids = input_ids[start_window:end_window]
    truncated_context = tokenizer.decode(truncated_input_ids, skip_special_tokens=True)

    return truncated_context


In [13]:
def preprocess_function(df, tokenizer, max_length=510):
    inputs = []

    # Loop through each row in the DataFrame
    for index, row in df.iterrows():
        question = row['question']
        context = row['context']
        answer_start = row['answer_start']
        answer_text = row['answers']
        answer_end = row['answer_end']

        # Tokenize the question and context, both with their respective prefixes
        question_tokenized = tokenizer(question, truncation=False)
        context_tokenized = tokenizer(context, return_offsets_mapping=True, truncation=False)
        total_length = len(question_tokenized['input_ids']) + len(context_tokenized['input_ids'])

        if total_length > max_length:
            # Call chunk_context to truncate context while retaining the answer
            truncated_context = chunk_context(question_tokenized, context_tokenized, answer_start, answer_end, tokenizer, max_length)
        else:
            truncated_context = context

        # Create the final input with both prefixes
        input_text = f"question: {question} context: {truncated_context}"
        inputs.append(input_text)

    # Tokenize the inputs w/o padding, which will be handled by Data Collator
    model_inputs = tokenizer(inputs, truncation=True, padding="False")

    # Tokenize the target answers (labels)
    labels = tokenizer([row['answers'] for _, row in df.iterrows()], max_length=75, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


In [14]:
# Apply preprocessing to train and validation datasets
tokenized_squad = preprocess_function(df, tokenizer, max_length=510)

In [15]:
# Save tokenizer
tokenizer.save_pretrained("./T5_tokenizer")

('./T5_tokenizer/tokenizer_config.json',
 './T5_tokenizer/special_tokens_map.json',
 './T5_tokenizer/spiece.model',
 './T5_tokenizer/added_tokens.json',
 './T5_tokenizer/tokenizer.json')

In [16]:
# Convert the tokenized data to a Hugging Face Dataset
tokenized_dataset = Dataset.from_dict({
    "input_ids": tokenized_squad["input_ids"],
    "attention_mask": tokenized_squad["attention_mask"],
    "labels": tokenized_squad["labels"]
})

In [17]:
# Split tokenized Hugging Face Datset into train and validation sets
train_val_split = tokenized_dataset.train_test_split(test_size=0.3)

# Create dataset dictionary to handle both train and validation sets
dataset_t5 = DatasetDict({
    'train': train_val_split['train'],
    'validation': train_val_split['test']
})

## Model Design and Training

In [19]:
# Load the pre-trained Flan-T5-Base model
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [20]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./flan-t5-finetuned-squad',  # Directory where the model checkpoints will be saved
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,  # Number of epochs for training
    evaluation_strategy="epoch",  # Evaluate at the end of every epoch
    save_steps=10_000,  # Save checkpoint every 10k steps
    save_total_limit=2,  # Only keep the last 2 checkpoints
    learning_rate=3e-5,
    warmup_steps=500,  # Warmup steps for learning rate scheduling
    weight_decay=0.01,  # Regularization
    logging_dir='./logs',  # Directory for logs
    logging_steps=500,  # Log every 500 steps
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
# Define a data collator for seq2seq tasks
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [22]:
# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_t5['train'],
    eval_dataset=dataset_t5['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator
)


In [23]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.025300,0.018490
2,0.021600,0.018066
3,0.019700,0.018019


TrainOutput(global_step=11499, training_loss=0.6919999873599796, metrics={'train_runtime': 5274.8128, 'train_samples_per_second': 34.875, 'train_steps_per_second': 2.18, 'total_flos': 8.608203039002112e+16, 'train_loss': 0.6919999873599796, 'epoch': 3.0})

In [24]:
# Save the fine-tuned model
model.save_pretrained("./flan-t5-finetuned-squad")
tokenizer.save_pretrained("./flan-t5-finetuned-squad")

('./flan-t5-finetuned-squad/tokenizer_config.json',
 './flan-t5-finetuned-squad/special_tokens_map.json',
 './flan-t5-finetuned-squad/spiece.model',
 './flan-t5-finetuned-squad/added_tokens.json',
 './flan-t5-finetuned-squad/tokenizer.json')

## Evaluation

In [25]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 0.01801919750869274,
 'eval_runtime': 208.6301,
 'eval_samples_per_second': 125.965,
 'eval_steps_per_second': 7.875,
 'epoch': 3.0}

In [96]:
# Load the dataset, specifying it as the test split
test_dataset = load_dataset('json', data_files={'test': '/content/drive/MyDrive/Colab Notebooks/Chatbot_Project/dev-v1.1.json'}, field='data')

# Verify that the dataset loaded correctly
print(test_dataset)

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})


In [101]:
def preprocess_test_questions(examples):
    # Extract questions from the nested structure
    questions = []

    # Access the paragraphs field (already a single example)
    for paragraph in examples['paragraphs']:
        # Extract questions from qas inside the paragraph
        for qa in paragraph['qas']:
            questions.append("question: " + qa['question'])

    # Tokenize the questions
    model_inputs = tokenizer(questions, max_length=512, truncation=True, padding="max_length")

    return model_inputs


In [102]:
# Tokenize the test split of the dataset
tokenized_test_dataset = test_dataset['test'].map(preprocess_test_questions, batched=False, remove_columns=["paragraphs"])


Map:   0%|          | 0/48 [00:00<?, ? examples/s]

In [111]:
# Function to generate predictions for the tokenized test dataset
def generate_predictions(tokenized_test_dataset):
    predictions = []

    # Loop through each example in the tokenized test dataset
    for i in range(len(tokenized_test_dataset)):
        # Access individual example
        example = tokenized_test_dataset[i]

        # If there are multiple sequences, iterate over each sequence
        for seq_index in range(len(example['input_ids'])):
            input_ids = torch.tensor(example['input_ids'][seq_index], dtype=torch.long).unsqueeze(0).to(device)  # Add batch dimension
            attention_mask = torch.tensor(example['attention_mask'][seq_index], dtype=torch.long).unsqueeze(0).to(device)  # Add batch dimension

            # Generate predictions using the model
            outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=100, num_beams=4, early_stopping=True)

            # Decode the predicted output into text
            decoded_pred = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Append the prediction to the list
            predictions.append(decoded_pred)

    return predictions

In [112]:
# Generate predictions for the test dataset
predictions = generate_predictions(tokenized_test_dataset)

In [115]:
# Extract multiple reference answers for each question from the test dataset
references = []

# Loop through the test dataset and collect the reference answers
for example in test_dataset['test']:  # Accessing the test split
    # Loop through paragraphs and then through qas
    for paragraph in example['paragraphs']:
        for qa in paragraph['qas']:
            # Extract all valid answers for the current question
            current_references = [answer['text'] for answer in qa['answers']]  # Collect all answers
            references.append(current_references)



In [116]:
# Tokenize predictions and references for BLEU
tokenized_predictions = [pred.split() for pred in predictions]  # Tokenize the predictions
tokenized_references = [[ref.split() for ref in refs] for refs in references]  # Tokenize each reference for each prediction


In [118]:
# Load BLEU metric
bleu_metric = load("bleu")
rouge_metric = load("rouge")

In [119]:
# Prepare the references by wrapping each reference in a list
bleu_references = [[ref] for ref in references]  # Each reference is wrapped in a list

# Compute the BLEU score without manual tokenization
bleu_results = bleu_metric.compute(predictions=predictions, references=bleu_references)
print("BLEU Score:", bleu_results['bleu'])

# Compute the ROUGE score directly on plain text
rouge_results = rouge_metric.compute(predictions=predictions, references=references)
print("ROUGE Scores:", rouge_results)

BLEU Score: 5.714529872265463e-05
ROUGE Scores: {'rouge1': 0.10350001215731952, 'rouge2': 0.0176990942202844, 'rougeL': 0.10243313352326026, 'rougeLsum': 0.10246072621504701}


In [128]:
# Number of examples to check
n_examples = 20  # Adjust this based on how many samples you want to check

# Keep track of index across paragraphs and questions
prediction_idx = 0

# Loop through paragraphs and questions
for paragraph_idx, paragraph in enumerate(test_dataset['test']):
    for qa_idx, qa in enumerate(paragraph['paragraphs'][0]['qas']):

        # Extract the question and context from the test dataset
        question = qa['question']
        context = paragraph['paragraphs'][0]['context']

        # Extract the corresponding prediction and reference answers
        prediction = predictions[prediction_idx]
        reference_answers = references[prediction_idx]  # This contains multiple references

        # Print out the aligned prediction, question, context, and references
        print(f"Example {prediction_idx + 1}:")
        print(f"Question: {question}")
        print(f"Context: {context}")
        print(f"Prediction: {prediction}")
        print(f"Reference Answers: {reference_answers}")
        print("=" * 50)

        # Move to the next prediction
        prediction_idx += 1

        # Stop after the desired number of examples
        if prediction_idx >= n_examples:
            break
    if prediction_idx >= n_examples:
        break


Example 1:
Question: Which NFL team represented the AFC at Super Bowl 50?
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
Prediction: Dallas Cowboys
Reference Answers: ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']
Example 2:
Question: Which NFL team repr

In [58]:
# Number of examples to check
n_examples = 10  # You can adjust this based on how many samples you want to check

# Loop through the first 'n_examples' predictions and references
for idx in range(n_examples):
    print(f"Example {idx + 1}:")
    print(f"Question: {df_test['question'][idx]}")
    print(f"Context: {df_test['context'][idx]}")
    print(f"Prediction: {predictions[idx]}")
    print(f"Reference: {references[idx]}")
    print("="*50)

Example 1:
Question: Which NFL team represented the AFC at Super Bowl 50?
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
Prediction: Denver Broncos
Reference: Denver Broncos
Example 2:
Question: Which NFL team represented the NFC at Super Bowl 50?
Context: Super

In [18]:
def build_conversation_history(history, new_turn, max_history_length=2):
    history.append(new_turn)
    # Limit history length to the last `max_history_length` turns to avoid exceeding token limits
    truncated_history = history[-max_history_length:]
    conversation_context = " ".join(truncated_history)
    return conversation_context


In [ ]:
def answer_question(question, context, tokenizer, model):
    input_text = f"question: {question} context: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Generate the answer using the fine-tuned model
    outputs = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)

    # Decode the output tokens to get the answer text
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Test the model with a sample question and context
question = "What is the capital of France?"
context = "France is a country in Europe. Its capital is Paris."
print(answer_question(question, context, tokenizer, model))
